In [1]:
# -*- coding: utf-8 -*-
import os
import random
import numpy as np
import librosa
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Permute, Reshape, Bidirectional, LSTM,
    Dropout, Embedding, Dense, MultiHeadAttention
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch

# =========================
# 재현성 고정
# =========================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# =========================
# 설정
# =========================
SAMPLE_RATE = 16000
N_MELS = 128
LSTM_UNITS = 128
DROPOUT = 0.3
MAX_TOKEN_LENGTH = 256
TOKEN_EMBED_DIM = 256
LR = 1e-4
EPOCHS = 100
THRESHOLD = 0.38  # 고정 임계치(원하시면 튜닝 함수로 대체 가능)

# =========================
# 타깃 라벨 및 유의어 매핑
# =========================
target_words = [
    "책가방", "아이", "현관", "신발", "강아지", "꼬리", "엄마", "소파", "뜨개질", "아빠", "신문",
    "텔레비전", "테이블", "로봇", "라디오", "창문", "나무", "참새", "책장", "책", "달력",
    "시계", "가족사진", "강아지집", "밥그릇", "식탁", "주전자", "찻잔", "과일", "액자"
]
NUM_LABELS = len(target_words)

synonym_map = {
    "티비": "텔레비전", "티브이": "텔레비전", "테레비": "텔레비전", "테레비/티비": "텔레비전",
    "테레비(텔레비전)": "텔레비전",
    "로보트": "로봇", "로보뜨(로봇)": "로봇",
    "신문지": "신문",
    "사진": "가족사진",
    "애기": "아이", "애": "아이", "애(아이)": "아이", "애들": "아이", "남자아이": "아이", "여자아이": "아이",
    "개": "강아지", "멍멍이": "강아지", "강아지/개": "강아지", "개/강아지": "강아지",
    "개집": "강아지집",
    "책꽂이": "책장",
    "잔": "찻잔"
}

def normalize_words(response_words):
    return [synonym_map.get(w, w) for w in response_words]

def get_label_vector(response_words, target_words):
    response_words = normalize_words(response_words)
    return np.array([1 if w in response_words else 0 for w in target_words], dtype=np.float32)

# =========================
# Whisper 로드
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = WhisperProcessor.from_pretrained("openai/whisper-medium", language="ko")
whisper_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium").to(device)
whisper_model.eval()

# 토크나이저 크기(+추가 토큰 포함). PAD 토큰을 별도로 부여
VOCAB_SIZE = len(processor.tokenizer)
PAD_ID = VOCAB_SIZE
VOCAB_SIZE_PLUS_PAD = VOCAB_SIZE + 1

@torch.no_grad()
def extract_token_ids_from_wav(wav_path: str):
    if not os.path.exists(wav_path):
        raise FileNotFoundError(f"WAV not found: {wav_path}")
    speech_array, _ = librosa.load(wav_path, sr=SAMPLE_RATE, mono=True)

    inputs = processor(
        speech_array,
        sampling_rate=SAMPLE_RATE,
        return_tensors="pt",
        language="ko"
    )
    input_features = inputs.input_features.to(device)

    # 한국어 전사 강제 프롬프트
    forced_ids = processor.get_decoder_prompt_ids(language="ko", task="transcribe")

    predicted_ids = whisper_model.generate(
        input_features,
        forced_decoder_ids=forced_ids,
        do_sample=False,
        max_new_tokens=MAX_TOKEN_LENGTH
    )
    token_ids = predicted_ids[0].cpu().numpy().astype(np.int32)

    # 길이 맞추기 (PAD_ID로 패딩)
    if token_ids.shape[0] < MAX_TOKEN_LENGTH:
        pad = np.full(MAX_TOKEN_LENGTH - token_ids.shape[0], PAD_ID, dtype=np.int32)
        token_ids = np.concatenate([token_ids, pad], axis=0)
    else:
        token_ids = token_ids[:MAX_TOKEN_LENGTH]

    return token_ids

# =========================
# 오디오 -> 멜스펙
# =========================
def load_wav_to_mel(wav_path: str):
    if not os.path.exists(wav_path):
        raise FileNotFoundError(f"WAV not found: {wav_path}")
    y, sr = librosa.load(wav_path, sr=SAMPLE_RATE, mono=True)
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=N_MELS)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    mel_norm = (mel_db + 80) / 80
    mel_norm = np.expand_dims(mel_norm, axis=-1)  # (128, time, 1)
    return mel_norm

def pad_mel_batch(mel_list):
    # time 축 기준 패딩
    max_time = max(m.shape[1] for m in mel_list)
    out = []
    for m in mel_list:
        pad_t = max_time - m.shape[1]
        if pad_t > 0:
            m = np.pad(m, ((0,0), (0,pad_t), (0,0)), mode='constant')
        out.append(m)
    return np.stack(out, axis=0)  # (B, 128, T, 1)

# =========================
# 직렬화 가능한 커스텀 레이어들
# =========================
from tensorflow.keras.layers import Layer
from keras.saving import register_keras_serializable

@register_keras_serializable(package="custom")
class AttentionPooling1D(Layer):
    """
    입력: (B, T, D)
    출력: (B, D)
    타임스텝 중요도를 학습하여 가중합으로 요약.
    """
    def __init__(self, units=128, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.proj = Dense(units, activation="tanh")
        self.score = Dense(1, use_bias=False)

    def call(self, x, mask=None):
        # x: (B, T, D)
        h = self.proj(x)           # (B, T, units)
        e = self.score(h)          # (B, T, 1)

        # (선택) 마스크가 들어오면 가려주기
        if mask is not None:
            mask = tf.cast(mask, tf.bool)          # (B, T)
            mask = tf.expand_dims(mask, axis=-1)   # (B, T, 1)
            minus_inf = tf.constant(-1e9, dtype=e.dtype)
            e = tf.where(mask, e, minus_inf)

        a = tf.nn.softmax(e, axis=1)               # (B, T, 1)
        return tf.reduce_sum(a * x, axis=1)        # (B, D)

    def get_config(self):
        cfg = super().get_config()
        cfg.update({"units": self.units})
        return cfg

@register_keras_serializable(package="custom")
class BuildCrossMask(Layer):
    """
    x1(query)의 길이 Tq와 토큰 실마스크(B, L)를 받아 (B, Tq, L) 어텐션 마스크 생성.
    """
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, inputs):
        q, tokmask = inputs                 # q: (B, Tq, D), tokmask: (B, L)  True=실토큰
        tokmask = tf.cast(tokmask, tf.bool)
        tokmask = tf.expand_dims(tokmask, axis=1)     # (B, 1, L)
        Tq = tf.shape(q)[1]
        tokmask = tf.tile(tokmask, [1, Tq, 1])        # (B, Tq, L)
        return tokmask

    def get_config(self):
        return super().get_config()

@register_keras_serializable(package="custom")
class TokenRealMask(Layer):
    """
    입력: (B, L) 정수 토큰 IDs
    출력: (B, L) bool, True=실토큰(패딩 아님)
    """
    def __init__(self, pad_id, **kwargs):
        super().__init__(**kwargs)
        self.pad_id = int(pad_id)

    def call(self, token_ids):
        return tf.not_equal(token_ids, self.pad_id)

    def get_config(self):
        cfg = super().get_config()
        cfg.update({"pad_id": self.pad_id})
        return cfg

# =========================
# 모델 정의
# =========================
def build_model():
    # 멜 입력: (B, 128, T, 1) -> (B, T, 128)
    mel_input = Input(shape=(N_MELS, None, 1), name='mel_input')
    x1 = Permute((2, 1, 3), name="permute_T1281")(mel_input)    # (B, T, 128, 1)
    x1 = Reshape((-1, N_MELS), name="reshape_T128")(x1)         # (B, T, 128)
    x1 = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True), name="mel_bilstm")(x1)
    x1 = Dropout(DROPOUT, name="mel_dropout")(x1)

    # 토큰 입력: (B, L)
    token_input = Input(shape=(MAX_TOKEN_LENGTH,), dtype='int32', name='token_input')
    x2 = Embedding(input_dim=VOCAB_SIZE_PLUS_PAD, output_dim=TOKEN_EMBED_DIM, name="tok_emb")(token_input)
    x2 = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True), name="tok_bilstm")(x2)
    x2 = Dropout(DROPOUT, name="tok_dropout")(x2)

    # 토큰 실마스크(True=실토큰) - 커스텀 레이어
    token_is_real = TokenRealMask(pad_id=PAD_ID, name="token_real_mask")(token_input)  # (B, L)

    # Cross-Attention 마스크 생성 (B, Tq, L)
    attn_mask = BuildCrossMask(name="build_cross_mask")([x1, token_is_real])

    # Cross-Attention (특성 256 = 64*4)
    cross_attn = MultiHeadAttention(num_heads=4, key_dim=64, name="cross_mha")
    x = cross_attn(query=x1, key=x2, value=x2, attention_mask=attn_mask)  # (B, T, 256)

    # Attention Pooling으로 요약
    x = AttentionPooling1D(units=128, name="attn_pool")(x)  # (B, 256)
    x = Dropout(DROPOUT, name="readout_dropout")(x)

    # 출력층
    output = Dense(NUM_LABELS, activation='sigmoid', name='output')(x)

    model = Model(inputs=[mel_input, token_input], outputs=output)
    metrics = [
        tf.keras.metrics.AUC(curve='PR', multi_label=True, name='PR-AUC'),
        tf.keras.metrics.AUC(curve='ROC', multi_label=True, name='ROC-AUC'),
    ]
    model.compile(optimizer=Adam(LR), loss='binary_crossentropy', metrics=metrics)
    return model

model = build_model()
model.summary()

# =========================
# 점수 리포트
# =========================
def score_and_report(preds, labels, target_words, threshold=THRESHOLD):
    for i, (pred, lab) in enumerate(zip(preds, labels)):
        bin_pred = (pred > threshold).astype(np.int32)
        acc = (bin_pred == lab.astype(np.int32)).mean()
        count = int(bin_pred.sum())

        print(f"\nSample {i} 결과")
        for w, p in zip(target_words, pred):
            print(f"  {w}: {p:.2f}")
        print(f"점수(임계값 {threshold:.2f}): {count} / {len(target_words)}")
        print(f"정확도: {acc*100:.2f}%")

# (옵션) 전역 임계치 튜닝 함수 - 필요 시 사용
def tune_threshold_global(preds, labels, start=0.05, stop=0.95, step=0.01):
    best_t, best_f1 = None, -1.0
    y_true = labels.reshape(-1)
    for t in np.arange(start, stop + 1e-9, step):
        y_pred = (preds > t).astype(np.int32).reshape(-1)
        tp = np.sum((y_pred == 1) & (y_true == 1))
        fp = np.sum((y_pred == 1) & (y_true == 0))
        fn = np.sum((y_pred == 0) & (y_true == 1))
        precision = tp / (tp + fp + 1e-8)
        recall    = tp / (tp + fn + 1e-8)
        f1        = 2 * precision * recall / (precision + recall + 1e-8)
        if f1 > best_f1:
            best_f1, best_t = f1, float(t)
    return best_t, best_f1

# =========================
# 데이터 (사용자 제공 리스트 붙여넣기)
# =========================
wav_paths = [
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1001\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1003\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1004\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\2006\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\2007\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3008\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3009\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3010\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3011\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\2014\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1015\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3017\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3018\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3019\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3020\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3022\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1023\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1024\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1025\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1026\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1027\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3028\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3029\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3030\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1032\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1033\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1034\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1035\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1036\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3037\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3038\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3039\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3040\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3041\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1042\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1043\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1045\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\4046\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\4047\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\4052\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\4053\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\4054\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\4055\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\4056\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\4057\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\5058\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\5059\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\5060\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\5061\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\5062\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\5063\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\5064\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\5065\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\5066\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\5067\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3070\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3071\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3074\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3075\CLAP_A\7\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3076\CLAP_A\7\p_1_0.wav",
]

response_words_list = [
    ["아빠", "사람", "신문", "뒤", "새", "3", "마리", "앞", "애기", "엄마", "소파", "뜨개질", "딸", "책", "지금", "오후", "시", "5", "월", "2", "일", "아들", "밖", "개", "1", "물", "가족", "사진", "4", "명", "카펫트", "중학교", "이름", "이길동"],
    ["시계", "달력", "엄마", "수건", "바느질", "아빠", "책", "새", "가족", "사진", "강아지", "집", "책꽂이", "애기", "학교", "문", "앞", "로보트", "옆", "얘기", "것", "바닥", "라디오"],
    ["가정집", "아이", "벽", "가족", "사진", "밖", "창문", "나무", "새", "안", "개집", "강아지", "책장", "책", "아빠", "신문", "아기", "앞", "엄마", "뜨개질", "옆", "시계", "3", "시", "오후", "달력", "5", "월", "2", "일", "탁자", "라디오", "로보트", "카펫트", "바닥"],
    ["아버지", "신문", "어머니", "소파", "뜨개질", "아들", "학교", "딸", "독서", "애기", "강아지", "가족", "5", "명", "새", "3", "마리", "월", "2", "일", "로보트", "카스테레오(카세트)", "전기장판", "책장", "책"],
    ["우리", "집", "거실", "신문", "책", "강아지", "앞", "장난", "아들", "바깥", "지금", "정원수", "참새", "집사람", "TV", "뜨개질", "서고", "위", "가족", "사진", "오늘", "5", "월", "2", "일", "하루", "일과", "마무리", "막내딸", "엄마", "옆", "그림책", "손녀딸/손녀", "할아버지", "재롱"],
    ["아들", "학교", "현관", "앞", "강아지", "아이", "가정", "집", "거실", "아빠", "신문", "뒤", "새", "3", "마리", "벽", "가족", "사진", "소파", "엄마", "뜨개질", "옆", "딸", "책", "탁자", "로봇", "장난감", "라디오", "벽", "시계", "3", "시", "달력", "5", "월", "2", "일", "방바닥", "노란색", "카펫", "애기", "책장"],
    ["오후", "3", "시", "집", "안", "아버지", "뜨개질", "신문", "아이", "라디오", "강아지", "때", "형"],
    ["아들", "딸", "뜨개질", "아버지", "옆", "창문", "신문", "책상", "위", "카세트테이프", "장난감", "방바닥", "아이", "외출", "남자", "강아지", "벽", "달력", "시계", "액자", "밑", "개집", "책장", "낙엽", "3", "개", "소파", "1"],
    ["소파", "위", "엄마", "사람", "딸", "2", "명", "책", "옷", "아버지/아빠", "책장", "옆", "신문", "소파", "앞", "책상", "장난감", "로봇", "라디오", "강아지", "집", "오늘", "5", "월", "일", "3", "시", "아들", "1", "개집", "가족", "사진", "뒤", "창문", "밖", "나무", "그루", "참새", "마리", "카페트", "아기"],
    ["애들", "아가", "아버지", "책", "엄마", "딸", "실", "할머니", "개", "가족", "사진", "강아지", "생각", "카세트", "나무", "옆", "새", "3", "마리"],
    ["집", "안", "거실", "아빠", "사람", "책", "엄마", "소파", "뜨개질", "여자", "애/아이", "테이블", "위", "라디오", "로보트", "장난감", "벽", "가족", "사진", "시계", "다음", "달력", "현관", "남자", "앞", "강아지", "창", "밖", "나무", "새", "카펫트", "책장"],
    ["5", "2", "월", "일", "날", "그림", "번째", "아이", "아들", "부모님", "인사", "강아지", "엄마", "주머니", "아버지", "마누라", "아기", "뒤", "집안"],
    [],
    ["학교", "신발", "강아지", "집", "엄마", "딸", "뜨개질", "실", "아빠", "신문", "애기", "앞", "라디오", "음악", "소리", "시간", "새벽", "3", "시", "날짜", "5", "월", "2", "일", "액자", "아들", "4", "명", "1", "책", "책장", "나비", "나무"],
    [],
    [],
    ["3", "시", "5", "월", "2", "일", "학교", "멍멍이", "엄마", "아빠", "신문", "뜨개", "동생", "책", "로보트", "카세트", "책상", "사진", "짹짹이", "나무"],
    ["현관", "엄마", "뜨개질", "아버지", "책", "아이", "라디오", "강아지", "집", "책장", "학생", "요거", "가족", "사진", "참새", "나무", "안방", "시계", "달력", "로보트", "책상", "탁자", "위", "어린이", "바닥", "카페트"],
    ["학생", "집", "가방", "학교", "강아지", "아빠", "책", "음악", "할머니", "뜨개질", "벽", "시계", "달력", "가족", "사진", "나무", "참새", "아이", "책꽂이", "라디오", "장난감", "거실", "소파", "식탁"],
    ["아빠", "신문", "책", "엄마", "뜨개질", "다음", "따님", "아들", "학교", "강아지", "가족", "사진", "자식", "3", "분", "하나", "창문", "너머", "새", "마리", "도서", "책", "비디오", "오늘", "5", "월", "2", "일", "3", "시", "어린이", "장난감"],
    ["엄마", "아이", "소파", "라디오", "아빠", "아기", "책", "어린이", "동화책", "개", "개집", "학생", "학교", "달력", "5", "월", "2", "일", "시계", "3", "시", "가족", "사진", "나무", "새"],
    ["그림", "속", "애(아이)", "숙제", "검사", "애기", "사람", "5", "명", "3"],
    ["아이", "엄마", "가게", "아버이(아버지)", "아가", "동물", "도깨비", "나무", "잎새/이파리", "1", "가족", "강아지", "집", "5", "월", "일", "9", "시"],
    ["오디오", "멍멍이", "아파(아빠)/아버지", "엄마", "책상"],
    ["책장", "뜨개질", "아이", "책", "공부", "학교", "준비", "5", "월", "2", "일", "고양이", "개", "신문", "애기", "참새", "텔레비", "사진"],
    ["나", "지금", "학교", "집", "중", "아빠/아버지", "신문", "어머니", "동생", "뜨개질", "안", "풍경", "우리", "가족", "사진", "밑", "고양이", "강아지", "옆", "책", "앞", "애", "막내", "소파", "라디오", "로보트", "장난감", "뒤", "시계", "다음", "달력", "오늘", "5", "월", "2", "일", "창문", "밖", "나무", "1", "그루", "거기", "여름", "매미"],
    ["엄마", "공부", "아빠"],
    ["가족", "시간", "사진", "강아지", "아이", "아빠", "엄마", "소파", "뜨개질", "동화책", "아들", "학교", "신발", "설명"],
    ["아기", "학생", "학교", "시계", "뜨개질", "엄마", "로보트", "강아지", "집", "라디오", "가족", "사진", "달력"],
    ["아들", "아빠", "딸", "옆", "뒤", "밑", "강아지", "다음", "할아버지", "새", "아침", "저녁", "잠"],
    ["아빠", "다음", "엄마", "동생", "애기", "강아지", "시계", "액자", "책", "테레비/티비", "로보뜨(로봇)"],
    ["가정집", "아부지(아버지)", "신문", "앞", "강아지", "1", "마리", "다음", "아들", "딸/딸아이", "바깥", "학교", "엄마", "뜨개질", "소파", "옆", "책", "벽", "중앙", "가족", "사진", "시계", "달력", "라디오", "로보트", "애들", "장난감", "카페트"],
    ["친구", "강아지", "책", "엄마", "뜨개", "아빠", "신문", "새", "가족", "사진", "책상", "시계", "3", "시", "5", "월", "2", "일", "전축"],
    ["거실", "엄마", "애기", "뜨개질", "아부지(아버지)", "책", "거실", "식탁", "라디오", "강아지", "집", "책꽂이", "사진", "액자", "나무", "그림", "시계", "달력", "연습장", "스카프"],
    ["5", "월", "2", "일", "날", "소년", "집", "엄마", "딸", "소파", "음악", "아빠", "바닥", "신문", "강아지/개", "1", "마리", "본인", "애기", "사진", "동생", "나", "여(여자)", "명", "바깥", "날씨", "새", "책", "테일(테이블)", "로봇", "밑", "양탄자", "권"],
    ["학생", "강아지", "엄마", "애(아이)", "바느질", "아버지", "학교", "지금", "그림", "새", "가족", "사진", "5", "월", "2", "일", "날짜", "오후", "3", "시", "옛날", "비디오", "테레비(텔레비전)"],
    ["집", "아버지", "신문", "아들", "엄마", "애기", "뜨개질", "딸", "책", "아버지", "강아지", "시계", "3", "시", "날짜", "5", "월", "2", "일", "액자", "가족", "사진", "창문", "나무", "참새", "라디오", "로보트", "탁자", "위"],
    ["(알 수 없는 웅얼거림)"],
    ["집", "침대", "아버지/아빠", "영화", "드라마", "TV", "딸내미", "책", "엄마", "아들내미", "인사", "자전거", "아가", "3", "5", "월", "2", "일", "시", "누나", "동생", "바깥", "그림", "차(자동차)", "택시", "마리", "오토바이", "음악"],
    ["테레비(텔레비전)", "시계", "책", "책꽂이"],
    ["엄마", "애기", "소파", "아빠", "강아지", "책장", "청년", "집", "할머니", "할아버지", "사진"],
    [],
    ["아들", "학교", "엄마", "아버지", "뜨개질", "애기", "책", "라디오", "노트", "강아지", "시계", "달력", "나무", "오리", "새"],
    ["1", "남자", "아이", "학교", "집", "아빠", "책", "강아지", "아기", "바닥", "엄마", "할머니", "여자", "나무", "밖", "새", "가족", "시계", "시간"],
    ["라디오", "악마", "아기", "책", "아빠"],
    [],
    ["남자", "퇴근", "집", "앞", "엄마", "뜨개질", "소년", "독서", "탁자", "장난감", "녹음기", "할아버지", "신문", "가족", "사진", "강아지", "꼬리", "아기", "참새", "카페트", "책장", "책", "30", "분", "2", "월"],
    ["책", "아저씨", "저녁", "강아지", "할머니", "딸", "숙제", "잡지", "라디오", "교회", "사진", "나무", "바깥", "잎새", "시계", "오늘", "5", "월", "2", "일", "카페트", "아래", "애기", "책장"],
    ["딸", "선생님", "애(아이)", "책", "라디오", "아기", "서점", "안경", "가족", "사진", "고양이", "산", "바다", "시계", "달력"],
    ["두꺼비"],
    ["집", "달력", "학교", "아부지(아버지)", "책", "멍멍이/강아지", "라디오", "로보트", "책장", "가족", "사진", "방", "거실", "그림", "밖", "새", "노트", "애기", "시계", "나", "엄마", "뜨개"],
    ["(알 수 없는 웅얼거림)"],
    ["가정", "아빠", "책", "엄마", "애(아이)", "옷", "실", "옛날", "뜨개", "바느질", "학교", "집", "안", "저녁", "휴식", "강아지", "애기", "표현", "라디오", "장난감", "로보트", "가족", "사진", "그림", "밖", "나무", "새", "시계", "달력", "거실", "생각"],
    ["엄마"],
    ["아줌마", "아이", "소파", "책", "뜨개질", "아저씨", "신문", "신발", "외출", "애기", "걸음", "책상", "사진", "그림", "라디오", "달력", "시계"],
    ["아들", "학교", "강아지", "아빠", "신문", "아가", "소파", "엄마", "뜨개질", "옆", "딸", "책", "식탁", "로보트", "라디오", "방바닥", "창", "밖", "나무", "새", "벽", "사진", "책장", "집", "달력", "시계"],
    ["(알 수 없는 웅얼거림)"],
    ["담요"],
    ["시계", "달력", "가족", "사진", "고양이", "개", "개집", "돗자리", "책", "끈", "딸", "아들", "아빠"],
    ["엄마", "딸", "소파", "뜨개질", "공부", "밖", "개/강아지", "애기", "혼자", "로보트", "라디오", "책상", "가족", "사진", "아버지", "신문", "새", "시계", "달력", "책꽂이", "집"],
]

# =========================
# 배치 구성
# =========================
mel_batch, token_batch, label_batch = [], [], []
for wav_path, response_words in zip(wav_paths, response_words_list):
    mel = load_wav_to_mel(wav_path)
    token_ids = extract_token_ids_from_wav(wav_path)
    label_vec = get_label_vector(response_words, target_words)

    mel_batch.append(mel)
    token_batch.append(token_ids)
    label_batch.append(label_vec)

mel_batch = pad_mel_batch(mel_batch)                 # (B, 128, T, 1)
token_batch = np.stack(token_batch, axis=0)          # (B, MAX_TOKEN_LENGTH)
label_batch = np.stack(label_batch, axis=0)          # (B, NUM_LABELS)

# =========================
# 데이터 분할 (8:2)
# =========================
X_mel_train, X_mel_test, X_token_train, X_token_test, y_train, y_test = train_test_split(
    mel_batch, token_batch, label_batch, test_size=0.2, random_state=SEED
)

# =========================
# 콜백: EarlyStopping
# =========================
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# =========================
# 학습
# =========================
batch_size = max(1, min(8, len(X_mel_train)))
history = model.fit(
    [X_mel_train, X_token_train],
    y_train,
    validation_data=([X_mel_test, X_token_test], y_test),
    epochs=EPOCHS,
    batch_size=batch_size,
    callbacks=[early_stop],
    verbose=1
)

# =========================
# 예측 & 리포트
# =========================
preds = model.predict([X_mel_test, X_token_test], verbose=0)
best_t, best_f1 = tune_threshold_global(preds, y_test, start=0.10, stop=0.90, step=0.01)
THRESHOLD = best_t if best_t is not None else THRESHOLD
print(f"\n[임계치 튜닝] 전역 최적 임계치 ≈ {THRESHOLD:.2f}")

score_and_report(preds, y_test, target_words, threshold=THRESHOLD)


c:\Users\ous37\my_project\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\ous37\my_project\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\ous37\my_project\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please update the gencode to avoid compatibili

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ mel_input           │ (None, 128, None, │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ permute_T1281       │ (None, None, 128, │          0 │ mel_input[0][0]   │
│ (Permute)           │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_T128        │ (None, None, 128) │          0 │ permute_T1281[0]… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ token_input         │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mel_bilstm          │ (None, None, 256) │    263,168 │ reshape_T128[0][… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tok_emb (Embedding) │ (None, 256, 256)  │ 13,277,696 │ token_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mel_dropout         │ (None, None, 256) │          0 │ mel_bilstm[0][0]  │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ token_real_mask     │ (None, 256)       │          0 │ token_input[0][0] │
│ (TokenRealMask)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tok_bilstm          │ (None, 256, 256)  │    394,240 │ tok_emb[0][0]     │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ build_cross_mask    │ (None, None, 256) │          0 │ mel_dropout[0][0… │
│ (BuildCrossMask)    │                   │            │ token_real_mask[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tok_dropout         │ (None, 256, 256)  │          0 │ tok_bilstm[0][0]  │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cross_mha           │ (None, None, 256) │    263,168 │ build_cross_mask… │
│ (MultiHeadAttentio… │                   │            │ tok_dropout[0][0… │
│                     │                   │            │ mel_dropout[0][0… │
│                     │                   │            │ tok_dropout[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attn_pool           │ (None, 256)       │     33,024 │ cross_mha[0][0]   │
│ (AttentionPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ readout_dropout     │ (None, 256)       │          0 │ attn_pool[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 30)        │      7,710 │ readout_dropout[… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 14,239,006 (54.32 MB)

 Trainable params: 14,239,006 (54.32 MB)

 Non-trainable params: 0 (0.00 B)

Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - PR-AUC: 0.2265 - ROC-AUC: 0.4008 - loss: 0.6926 - val_PR-AUC: 0.2708 - val_ROC-AUC: 0.3715 - val_loss: 0.6919
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - PR-AUC: 0.2369 - ROC-AUC: 0.4274 - loss: 0.6908 - val_PR-AUC: 0.2725 - val_ROC-AUC: 0.3758 - val_loss: 0.6902
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - PR-AUC: 0.2590 - ROC-AUC: 0.4390 - loss: 0.6883 - val_PR-AUC: 0.2746 - val_ROC-AUC: 0.3814 - val_loss: 0.6876
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - PR-AUC: 0.2784 - ROC-AUC: 0.4678 - loss: 0.6840 - val_PR-AUC: 0.2862 - val_ROC-AUC: 0.4037 - val_loss: 0.6826
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - PR-AUC: 0.2633 - ROC-AUC: 0.4563 - loss: 0.6754 - val_PR-AUC: 0.3115 - val_ROC-AUC: 0.4306 - val_loss: 0.6716
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - PR-AUC: 0.2689 - ROC-AUC: 0.4773 - loss: 0.6535 - val_PR-AUC: 0.3026 - val_ROC-AUC: 0.4267 - val_loss: 0.6441
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s

In [ ]:
MODEL_PATH = "multilabel_whisper_attn_medium.keras"
model.save(MODEL_PATH)